In [1]:
import os
import gc
import cv2
import time
import tqdm
import random
import collections
import numpy as np
import pandas as pd
from PIL import Image
from functools import partial
import matplotlib.pyplot as plt
import tqdm.notebook as tq
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from multiprocessing import Pool
import shutil


import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

from torch.nn import ConvTranspose2d
from torch.nn import Conv2d
from torch.nn import MaxPool2d
from torch.nn import Module
from torch.nn import ModuleList
from torch.nn import ReLU
from torchvision.transforms import CenterCrop
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam

from torchmetrics import JaccardIndex
import threading

import json

# import segmentation_models_pytorch as smp
# from segmentation_models_pytorch.encoders import get_preprocessing_fn
# preprocess_input = get_preprocessing_fn('resnet34', pretrained='imagenet')

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [3]:
import ptlflow
from ptlflow.utils import flow_utils
from ptlflow.utils.io_adapter import IOAdapter

ERROR: torch_scatter not found. CSV requires torch_scatter library to run. Check instructions at: https://github.com/rusty1s/pytorch_scatter


In [4]:
root_path = "/home/thiago/Workspace/motion-segmentation/datasets/WHUVID"
# sequences = ["01", "02", "03", "17", "18", "19", "20", "22", "23", "24", "25", "30", "31", "32"]
sequences = ["02"]
# sequences = ["01"]

In [5]:
def image_list_from_file(path):
    with open(path) as f:
        lines = f.readlines()
    lines = [line.rstrip() for line in lines]
    # remove first 3 (header)
    return lines[3:]

def get_next_image_path(image_list, image_path, diff):
    # find index of image_path in image_list
    # remove everything before "cam0"
    relative_path = image_path[image_path.find("cam0"):]
    # get everything before cam0
    root = image_path[:image_path.find("cam0")]
    idx = image_list.index(relative_path)
    # return None if image_path is one of the last images
    image_diff = diff
    if idx >= (len(image_list) - image_diff):
        return None
    next_image_path = os.path.join(root, image_list[idx+image_diff])
    # check if image exists
    if not os.path.exists(next_image_path):
        return None
    return next_image_path

def get_files(root_path, cam):
    images = []
    image_list = image_list_from_file(os.path.join(root_path, "other_files/image.txt"))
    images_path = os.path.join(root_path, cam)
    for root, dirs, files in os.walk(images_path):
        for file in files:
            full_path = os.path.join(root, file)
            next_image = get_next_image_path(image_list, full_path, 10)
            if full_path.endswith(".png")and next_image is not None:
                # prevent !_src.empty()
                # img = cv2.imread(full_path)
                # if img is None:
                #     continue
                images.append(full_path)
    return images

In [6]:
images = []
for seq in sequences:
    path = os.path.join(root_path, seq)
    images += get_files(path, "cam0")
    flow_dir = os.path.join(path, "flow_v7", "cam0")
    if not os.path.exists(flow_dir):
        os.makedirs(flow_dir)

In [7]:
len(images)

8621

In [8]:
def get_path_of_flow_image(image_path):
    # remove everything before "cam0"
    relative_path = image_path[image_path.find("cam0"):]
    # get everything before cam0
    root = image_path[:image_path.find("cam0")]
    flow_path = os.path.join(root, "flow_v7", relative_path)
    return flow_path

In [9]:
model = ptlflow.get_model('dicl', pretrained_ckpt='kitti').cuda()
# model = ptlflow.get_model('raft', pretrained_ckpt='kitti').cuda()

/home/thiago/Workspace/motion-segmentation/src/.venv/lib/python3.12/site-packages/pytorch_lightning/utilities/parsing.py:208: Attribute 'loss_fn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_fn'])`.


In [10]:
images.sort()

In [11]:
def flow_single_image(path):
    try:
        flow_path = get_path_of_flow_image(path)
        # extract dir and create if not exists
        # flow_dir = os.path.dirname(flow_path)
        # recursively remove flow_dir
        # if not os.path.exists(flow_dir):
        #     os.makedirs(flow_dir, exist_ok=True)
        # get next image name
        # root_path is all before "cam0"
        root_path = path[:path.find("cam0")]
        image_list = image_list_from_file(os.path.join(root_path, "other_files/image.txt"))
        next_image_path = get_next_image_path(image_list, path, 10)
        # compute flow
        if next_image_path is None:
            return
        prev_image = cv2.imread(path)
        next_image = cv2.imread(next_image_path)

        height, width, _ = prev_image.shape
        factor = 1.5
        target_height, target_width = round(height//factor), round(width//factor)
        # target_height, target_width = height, width
        prev_image = cv2.resize(prev_image, (target_width, target_height))
        next_image = cv2.resize(next_image, (target_width, target_height))

        imgs = [prev_image, next_image]
        io_adapter = IOAdapter(model, imgs[0].shape[:2])
        inputs = io_adapter.prepare_inputs(imgs)
        inputs['images'] = inputs['images'].cuda()

        predictions = model(inputs)
        flows = predictions['flows']
        flow_rgb = flow_utils.flow_to_rgb(flows, flow_max_radius=150)
        # Make it a numpy array with HWC shape
        flow_rgb = flow_rgb[0, 0].permute(1, 2, 0)
        flow = flow_rgb.detach().cpu().numpy()
        # OpenCV uses BGR format
        flow = cv2.cvtColor(flow, cv2.COLOR_RGB2BGR)
        # to 0-255
        flow = (flow * 255).astype(np.uint8)
        # return to original size
        flow = cv2.resize(flow, (width, height))
        # save flow image
        cv2.imwrite(flow_path, flow)
        
        
        # free memory
        del prev_image
        del next_image
        del flow
        del flow_rgb
        del flows
        del predictions
        del inputs
        del imgs
        gc.collect()
    except Exception as e:
        print(f"Skipping {path} due to {e}\n")
# using pool, with tqdm
# with Pool(28) as p:
#     list(tq.tqdm(p.imap(flow_single_image, images), total=len(images)))\

# without pool
for i, image in tq.tqdm(enumerate(images), total=len(images)):
    flow_single_image(image)

  0%|          | 0/8621 [00:00<?, ?it/s]